# Fx Forward Pricing

This notebook will run through the following business use cases :
* [Pricing a set of Fx Forwards using the internal LUSID "Discounting" model](#pricing_bond)
* [Booking a set FX forward instrument transactions and analysing it's impact on holdings the over the life of the contract.
* [Valuing a portfolio made up of an Fx forward position with fluctuating market data.](#pricing_bond_portfolio)

<br>

In doing so we'll cover the following LUSID concepts :
* [Defining a LUSID internal representation of an FX forward instrument based on user provided parameters.](#bond_definition)
* [Supplying both simple and complex market data structures through the Quotes and the StructureMarketData store.](#structured_market_data)
* [Configuring recipes to value our FX forward in LUSID making use of the structured market data (GBP and USD OIS Yield Curves).](#recipe_configuration)
* [Running inline aggregations that value a unitised FX forward.](#accrued_interest)
* Creating custom transaction types and custom sides to properly model the life of an Fx Forward
* Running aggregations that value a portfolio containing FX forwards.

<br>

For this notebook example we'll work the following set of GBP/USD Forwards with 3Y, 5Y and 10Y maturities:

* Trade Date : 02 Aug 2020
* Maturity Dates :
    * 02 Aug 2023 (3Y)
    * 02 Aug 2025 (5Y)
    * 02 Aug 2030 (10Y)
* Spot Rate : 1.2508
* Domestic Amount : 100,000,000
* Foreign Amounts :
    * 125,710,000 (3Y)
    * 126,485,000 (5Y)
    * 130,430,000 (10Y)


## Setup LUSID and LUSID API objects.

In [5]:
import os
from datetime import datetime, timedelta

import lusid
import pandas as pd
import pytz
from IPython.core.display import display
from lusid import models
from lusid.utilities import ApiClientFactory
from lusidjam import RefreshingToken
from lusidtools.cocoon.transaction_type_upload import (
    create_transaction_type_configuration,
)
from lusidtools.cocoon.cocoon_printer import (
    format_portfolios_response,
)
from lusidtools.jupyter_tools.stop_execution import StopExecution


# Authenticate our user and create our API client
from lusidtools.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename=secrets_path,
    app_name="LusidJupyterNotebook",
)

print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

# Setup the apis we'll use in this notebook:
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
structured_market_data_api = api_factory.build(lusid.api.StructuredMarketDataApi)
structured_result_data_api = api_factory.build(lusid.api.StructuredResultDataApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
system_configuration_api = api_factory.build(lusid.api.SystemConfigurationApi)

# Setup the scope we'll use in this notebook:
scope = "fx-forward-pricing-nb"

LUSID Environment Initialised
LUSID SDK Version:  0.6.4778.0


In [6]:
# Settings and utility functions to display objects and responses more clearly.
pd.set_option('float_format', '{:f}'.format)
def aggregation_result_to_dataframe(aggregation_results):
    df = pd.DataFrame(aggregation_results, columns = ['Name', 'Effective At', 'Value'])
    df['Effective At'] = df['Effective At'].dt.strftime('%d %b %Y')
    return df

def display_holdings_summary(response, effective_at):
    # inspect holdings response for today
    hld = [i for i in response.values]

    names = []
    #amount = []
    units = []
    holding_types = []
    cost = []
    settled_units = []


    for item in hld:

        names.append(item.properties["Instrument/default/Name"].value.label_value)
        #amount.append(item.cost.amount)
        units.append(item.units)
        holding_types.append(item.holding_type)
        settled_units.append(item.settled_units)
        cost.append(item.cost)


    data = {"names": names, "effectiveAt" : effective_at, "units": units, "settled_units": settled_units, "holding types": holding_types, "cost" : cost}

    summary = pd.DataFrame(data=data)
    return summary

## Define our Fx Forward Instruments

We'll start by defining our forwards using LUSID's internal representation of an Fx forward instrument. For a detailed
breakdown the internal representation take a look at the "FxForwardInstrument" section of the [LUSID API Swagger Specification](https://www.lusid.com/api/swagger/index.html)

### Initialise FxForward Parameters

Let's setup the basic parameters required for our set of Fx forwards:

In [7]:
trade_date = datetime(2020, 6, 1, tzinfo=pytz.utc)
#trade_date = datetime.today().replace(tzinfo=pytz.utc)
maturity_dates = {
    '1W' : datetime(2020, 6, 8, tzinfo=pytz.utc),
    '3Y' : datetime(2023, 6, 1, tzinfo=pytz.utc),
    '5Y' : datetime(2025, 6, 1, tzinfo=pytz.utc),
    '10Y': datetime(2030, 6, 1, tzinfo=pytz.utc)
}
spot_rate =  1.2508
dom_amount = 100000000
fgn_amounts = {
    '1W' : 125090000,
    '3Y' : 125710000,
    '5Y' : 126485000,
    '10Y': 130430000
}

### Create FxForwardInstrument Definitions

We can use those parameters to construct a LUSID representation of an FxForward. In our case we'll define a set made up of a
3Y, 5Y and 10Y GBP/USD Fx Forward.

In [8]:
def create_fx_forward_instrument_definition(dom_amount, fgn_amount, spot_rate, start_date, maturity_date):
    return models.FxForwardInstrument(
            dom_amount=dom_amount,
            fgn_amount=-fgn_amount,
            fgn_ccy="USD",
            ref_spot_rate=spot_rate,
            start_date=start_date.isoformat(),
            maturity_date=maturity_date,
            dom_ccy="GBP",
            instrument_type="FxForward")

gbpusd_fx_fwd_1w_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['1W'], spot_rate, trade_date, maturity_dates['1W'])
gbpusd_fx_fwd_3y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['3Y'], spot_rate, trade_date, maturity_dates['3Y'])
gbpusd_fx_fwd_5y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['5Y'], spot_rate, trade_date, maturity_dates['5Y'])
gbpusd_fx_fwd_10y_instr_def = create_fx_forward_instrument_definition(dom_amount, fgn_amounts['10Y'], spot_rate, trade_date, maturity_dates['10Y'])

## Supply Market Data

Before we can price our Fx forwards we need to ensure LUSID has the required FX and interest rates available. This allows us
to showcase two ways of getting the market data we need to price an FX forward into LUSID. The simpler method for the FX rates
can simply be upserted as quotes but the more core complex structure of the FGPB and USD yield curves required the use of the
Structured Market Data store.

Let's start by upserting the GBP/USD FX rates for the trade data and a few subsequent dates we'll use to price the FX Forwards.
So we can see some movement in the forward prices we'll mock a drop in the spot rate on June 2nd followed by a sharp rise on June 3rd. When providing market
data to LUSID we also need to identify the supplier of the data and it's scope. The supplier is required further in the
notebook when we instruct LUSID how to source data for a particular aggregation.

### Upsert Spot Fx Rates

In [31]:
market_data_scope = 'fx-forward-pricing-nb-market-data'
market_supplier = 'Lusid'

t_p_1 = datetime(2020, 6, 2, tzinfo=pytz.utc)
t_p_2 = datetime(2020, 6, 3, tzinfo=pytz.utc)


def upsert_gbp_usd_fx_rate(rate, effective_at):
    upsert_quote_request = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="GBP/USD",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    # we need to insert the USD/GBP quote as well for use later in the notebook
    # when we're booking the sell leg of our forward as a transaction
    upsert_quote_request_inverse = models.UpsertQuoteRequest(
        quote_id=models.QuoteId(
            quote_series_id=models.QuoteSeriesId(
                provider=market_supplier,
                instrument_id="USD/GBP",
                instrument_id_type='CurrencyPair',
                quote_type='Price',
                field='mid'),
            effective_at=effective_at),
        metric_value=models.MetricValue(
            value=1/rate,
            unit='rate'
        ),
        lineage='FxDataVendorABC')

    response_gbp_usd = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"gbp-usd-01": upsert_quote_request})

    response_usd_gbp = quotes_api.upsert_quotes(
            scope=market_data_scope,
            request_body={"usd-gbp-01": upsert_quote_request_inverse})

    if response_gbp_usd.failed or response_usd_gbp.failed:
        raise StopExecution(f"Failed to upload currency pairs:{response_gbp_usd.failed} or {response_usd_gbp.failed}")

    display(f"GBP/USD @ {rate} for {effective_at} uploaded to quote store.")
    display(f"USD/GBP @ {1/rate} for {effective_at} uploaded to quote store.")


upsert_gbp_usd_fx_rate(spot_rate, trade_date)
# drop in fx rate
upsert_gbp_usd_fx_rate(spot_rate - 0.0080, t_p_1)
# increase in fx rate
upsert_gbp_usd_fx_rate(spot_rate + 0.0080, t_p_2)


'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2428 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.804634695848085 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.2588 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7944073721004131 for 2020-06-03 00:00:00+00:00 uploaded to quote store.'

### Interest Rate Curves

As well as the FX rate to price our Forwards we also need to supply the interest rate curves for our domestic and foreign currencies. LUSID supports
storage of complex market data structures through the [Structured market data]("https://support.finbourne.com/how-do-i-store-and-tr")
store. When upserting the interest rate curves we use a StructuredMarketDataId to not onl uniquely identify the curve but also provide
the supporting information required by the aggregation engine to correctly resolve the curves during the execution of the FX forward valuation.

In [26]:
def upsert_interest_rate_curve(ir_curve_json, scope, effective_at, market_asset):

    # provide the structured data file source and it's document format
    structured_market_data = models.StructuredMarketData(document_format="Json", version="1.0.0",
                                                name="DFEUROISCurve", document=ir_curve_json)


    # create a unique identifier for our OIS yield curves
    structured_id = models.StructuredMarketDataId(provider="Lusid",price_source=None,
                                                 lineage="CurveVendorABC", effective_at=effective_at,
                                                 market_element_type="ZeroCurve",
                                                 market_asset=market_asset)

    upsert_request = models.UpsertStructuredMarketDataRequest(market_data_id=structured_id,
                                                          market_data=structured_market_data)

    # https://www.lusid.com/docs/api#operation/UpsertStructuredMarketData
    response = structured_market_data_api.upsert_structured_market_data(
        scope=scope,
        request_body={market_asset : upsert_request}
    )

    if response.failed:
        raise StopExecution("Failed to upload interest rates curve {response.failed}")

    print(f"{market_asset} interest rate curve uploaded into scope={scope}")


def load_ois_curve_json(ccy, shift=None):
    ois_curve_path = f"data/{ccy}OIS50_shift.json" if shift else f"data/{ccy}OIS50.json"
    with open(ois_curve_path, "r") as ir_curve_json:
        return ir_curve_json.read()

ois_curve_gbp_json = load_ois_curve_json('GBP')
upsert_interest_rate_curve(ois_curve_gbp_json, market_data_scope, trade_date, "GBP/GBPOIS")

ois_curve_usd_json = load_ois_curve_json('USD')
upsert_interest_rate_curve(ois_curve_usd_json, market_data_scope, trade_date, "USD/USDOIS")

GBP/GBPOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data
USD/USDOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data


## Defining our Fx Forward Valuation

With our Fx forward instruments now defined we can move onto valuing them via LUSID's aggregation engine. But before we
can execute the aggregation we need to configure a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used")
instructing LUSID on how to value our Fx forwards. Specifically we need to select the Fx Forward pricing model to use and identify
where the aggregation can resolve the market data the model needs.

### Selecting a Pricing Model

The pricing model we wish to use is passed in through defining a PricingContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html") under "PricingContext" for a detailed
description of the parameters. For this notebook we'll use the LUSID supplied "Discounting" model:

In [11]:
def create_pricing_context():
    vendor_model_rule = models.VendorModelRule(
        supplier="Lusid",
        model_name="Discounting",
        instrument_type="FxForward",
        parameters="{}")

    return models.PricingContext(
        model_rules=[vendor_model_rule]
    )

pricing_context = create_pricing_context()

### Selecting the Market Data

We can instruct LUSID on where to resolve market data required for pricing our Fx forwards through defining a MarketContext. See the [Swagger spec]("https://www.lusid.com/api/swagger/index.html#model-MarketContext")
under "MarketContext" for a detailed description of the parameters. Recall that when we upserted our quote we passed in a supplier and a scope. One of
the powerful features in LUSID is the ability to run valuations against market data from different suppliers, or even define market data retrieval rules
to fallback to suppliers should your primary supplier not have the required data.

However our example is trivial as we only require the spot rate we loaded in earlier. So we'll define a simple market context :

In [12]:
def create_market_context():
    return models.MarketContext(
        # set rules for where we should resolve our rates data. In our case the interest rate curves we inserted into
        # the structured market result data store.
        market_rules=[
            models.MarketDataKeyRule(
                key="Rates.*.*",
                data_scope=market_data_scope,
                supplier=market_supplier,
                quote_type='Rate',
                field='Mid')
        ],
        # control default options for resolving market data. In our case simply default to the LUSID market_supplier
        # and market data scope we defined earlier.
        options=models.MarketOptions(
            default_supplier=market_supplier,
            default_scope=market_data_scope)
    )

    return market_context

market_context = create_market_context()

### Configure our FX Forward Recipe

Now that we've defined what pricing model to use and where to source the market data we can bring those instructions
together in a [Recipe]("https://support.finbourne.com/what-is-a-lusid-recipe-and-how-is-it-used"):

In [13]:
def create_fx_forward_pricing_recipe():
    return models.ConfigurationRecipe(
            scope=scope,
            code="fx-forward-pricing-recipe",
            description="Price Fx Forwards using LUSID internal model",
            market=market_context,
            pricing=pricing_context
        )

fx_forward_pricing_recipe = create_fx_forward_pricing_recipe()

## Price the FX Forwards

Let's summarise our current state:
 * We've defined a set of GBP/USD FX forwards for maturity in 3Y, 5Y and 10Y.
 * We've loaded in the spot GBP/USD rates for the dates we're going to value our forwards against.
 * We've setup our Recipe that configures how we would like to price our bond and where to source our required market data.

As we have no existing positions in the FX forward booked against a portfolio we'll run our aggregation on an inlined portfolio.
That is a portfolio made up of a set of weighted instruments only. In our case each request will be made up of only one instrument which
will be the FX forward we're valuing.

### Run an Aggregation to Price our Forwards

In [14]:
def run_fx_forward_pricing_aggregation(maturity, effective_at, fx_forward_instrument_definition, pricing_recipe):
    # setup weighted instrument (only our gilt definition)
    fx_forward = models.WeightedInstrument(quantity=1, instrument=fx_forward_instrument_definition, holding_identifier=f"{maturity}-holding")

    # create our aggregation request made up of our recipe and the metrics we would like to calculate (Bond PV)
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at,
        inline_recipe=pricing_recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV', op='Value'),
        ]
    )

    # As we're running an inline aggregation we must wrap our original aggregation request with an inline aggregation
    # request and pass in our weighted instruments
    inline_aggregation_request = models.InlineAggregationRequest(
        request=aggregation_request, instruments=[fx_forward]
    )

    # https://www.lusid.com/docs/api#operation/GetAggregationOfWeightedInstruments
    return api_factory.build(lusid.api.AggregationApi).get_aggregation_of_weighted_instruments(
        market_data_scope, inline_aggregation_request=inline_aggregation_request)


Let's start by running the 5Y forward valuation across the three dates for which we upserted spot GBP/USD quotes. Recall the spot rate quotes were 1.2508 for
trade date, which then dropped by 0.0080 on day two and subsequently then increased by 0.0160 on day 3.

In [27]:
result_t = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_one = run_fx_forward_pricing_aggregation('5Y', t_p_1, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_t_plus_two = run_fx_forward_pricing_aggregation('5Y', t_p_2, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 5Y Forward PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', t_p_1, result_t_plus_one.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', t_p_2, result_t_plus_two.data[0]['Holding/default/PV']]
])

,Name,Effective At,Value
0,GBP/USD 5Y Forward PV,01 Jun 2020,-0.011177
1,GBP/USD 5Y Forward PV,02 Jun 2020,-0.017654
2,GBP/USD 5Y Forward PV,03 Jun 2020,-0.004782


Let's now take a slightly different perspective and look at the valuations across the Forward curve:

In [52]:
result_3y = run_fx_forward_pricing_aggregation('3Y', trade_date, gbpusd_fx_fwd_3y_instr_def, fx_forward_pricing_recipe)
result_5y = run_fx_forward_pricing_aggregation('5Y', trade_date, gbpusd_fx_fwd_5y_instr_def, fx_forward_pricing_recipe)
result_10y = run_fx_forward_pricing_aggregation('10Y', trade_date, gbpusd_fx_fwd_10y_instr_def, fx_forward_pricing_recipe)

aggregation_result_to_dataframe([
    ['GBP/USD 3Y Forward PV', trade_date, result_3y.data[0]['Holding/default/PV']],
    ['GBP/USD 5Y Forward PV', trade_date, result_5y.data[0]['Holding/default/PV']],
    ['GBP/USD 10Y Forward PV', trade_date, result_10y.data[0]['Holding/default/PV']]
])


,Name,Effective At,Value
0,GBP/USD 3Y Forward PV,01 Jun 2020,-0.002024
1,GBP/USD 5Y Forward PV,01 Jun 2020,-0.006160
2,GBP/USD 10Y Forward PV,01 Jun 2020,-0.032075


## Adding FX Forwards to our Portfolio

TODO introcude working with portfolio:
    Discuss need to properly book transactions on open and close
    Discuss Pricing



Up until now we've only priced our Fx Forwards as unitised instruments. Let's now take it a step further and build a simple portfolio made up of
an actual FX forward position we decide to enter with a counterparty and see it through till maturity. Here's a quick preview of what we aim to do :
* Setup a portfolio and book a transaction entering into an FX Forward position
* Simulate market movements in fx and interest rates during the life of the position and run valuations to view the impact on our portfolio PV.
* Book a transaction on maturity closing the forward and reflecting the cash movements.
* Analyse the the evolution of our portfolio holdgins over the life of the contract.



### Setting up our Portfolio

We'll firstly setup the portfolio that will hold our FX forward positions:

In [17]:
portfolio = "fxfwd-portfolio-01"

def create_portfolio(scope, portfolio_code, portfolio_name, portfolio_ccy):
    pfs = [[portfolio_code, portfolio_name, portfolio_ccy]]
    pf_df = pd.DataFrame(pfs, columns=['portfolio_code', 'portfolio_name', 'base_currency'])

    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": "base_currency",
        },
        "optional": {"created": "$2020-01-01T00:00:00+00:00"},
    }
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
        sub_holding_keys=[],
    )
    succ, failed = format_portfolios_response(result)

    if not failed.empty:
        raise StopExecution(failed)

    return succ

create_portfolio(scope, portfolio, portfolio, "GBP")



,successful items
0,fxfwd-portfolio-01


### Upserting our FX Forward Instrument

Recall when valuing our unitised Fx Forwards earlier we ran an "inline" aggregatoin that did not require us to hold a position
in the forward. However we now no longer want to value the unitised FX forward but actually value our position in the forward. To do
so we'll no longer run an inline aggregation but instead a portfolio aggregation.

As the valuation method and market data sources aren't changing we can simply reuse our "FX Forward Discounting" recipe we constructed
earlier. However we will now be booking a transaction against our FX Forward instrument we need an instrument identifier that our transactoin
can book against. Therefore we now need to actually upsert the FX forward and store it in LUSID.

Our portfolio will only include a position in the GBP/USD 1W forward:

In [18]:
def upsert_fx_fwd_instrument(instrument_id, instrument_name, fx_fwd_definition):
    bond_instrument_request = {instrument_id: models.LusidInstrumentDefinition(
        # instrument display name
        name=instrument_name,
        # unique instrument identifier
        identifiers={"ClientInternal": models.InstrumentIdValue(instrument_id)},
        # our gilt instrument definition
        definition=fx_fwd_definition
    )}
    # Note we're using upsert_lusid_instrument and not upset_instrument as we're creating an instrument based
    # on a user defined instrument definition and not the base LUSID instruments.
    return instruments_api.upsert_lusid_instruments(bond_instrument_request)

# the identifier we'll use to reference the instrument when booking transactions.
gbpusd_fx_fwd_1w_instr_client_id = "gbp_usd_fwd_1w_12509"
instrument_creation_response = upsert_fx_fwd_instrument(gbpusd_fx_fwd_1w_instr_client_id, "GBP/USD 1W @ 1.2509", gbpusd_fx_fwd_1w_instr_def)


### Setting up our FX Forward Transaction

Note : The next section will make use of defining transaction types and sides. For a recap of those concepts then please review
the following KB artciles and notebooks:
* [Configuring Transaction Types](https://support.finbourne.com/configuring-transaction-types)
* [What is a Side in LUSID](https://support.finbourne.com/what-is-a-side-in-lusid)
* [NB : [Generating holdings with the movements engine in LUSID](https://github.com/finbourne/sample-notebooks/blob/master/examples/use-cases/ibor/Generating%20holdings%20with%20the%20movements%20engine%20in%20LUSID.ipynb)

We'd like to open a Long position in the FX forward GBP/USD 1W @ 1.2509 with the spot @ 1.2508. Our position is to buy 10,000 GBP
and sell 12,509 USD on maturity.

Our FX Forward is defined as an instrument we'd like to buy. However the default "Buy" transaction type for LUSID is not suitable
for a forward. The reason being is that although we would like to increase our "stock" holdings in the instrument we don't at this
stage want to reflect any cash movement as none has been exchanged. So instead we define our own transaction type.

##### Creating a Transaction Type (Opening an FX Forward Position)

Let's define a transaction that simply increases our "stock" holding in the FX Forward instrument:

In [19]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdOpen",
        description="Open an FX Forward position",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=1
        )
    ],
)



#### Upserting our transaction

We're now ready to book a transaction in our FX forward. At this point it's worth remembering that when we were valuing
our FX forward earlier, we were doing so on a unitised basis. Our transaction is where we would book the actual notionals
involved in our contract.

Recall we've agreed to Buy 10,000GBP and Sell 12,509 USD on maturity:

In [21]:
# our instrument is "unitised" we'd like to open 10,000 units to reflect our notional
units = 10000
transaction_price = 1.2509  # spot + 3y fwd spread
consideration = 10000

gilt_transaction_request = models.TransactionRequest(
    transaction_id="fx_fwd_1w_b_001",
    # our custom transaction type we setup
    type="FxFwdOpen",
    # the id we setup for our 1W forward that we upserted earlier
    instrument_identifiers={"Instrument/default/ClientInternal": gbpusd_fx_fwd_1w_instr_client_id},
    transaction_date=trade_date.isoformat(),
    settlement_date=maturity_dates['1W'],
    units=units,
    transaction_price=models.TransactionPrice(price=transaction_price, type="Price"),
    total_consideration=models.CurrencyAndAmount(amount=consideration, currency="GBP"),
    exchange_rate=1,
    transaction_currency="GBP"
)

response = api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                     code=portfolio,
                                                                                     transaction_request=[gilt_transaction_request])

#### View our Holdings

We've now taken a position in the FX Forward and can see it reflected as such in our holdings :

In [22]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

display_holdings_summary(holdings_response_trade_date, trade_date)


,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"


### Valuing our Portfolio

Let's now run a valuation of the portfolio remembering that we're using the same recipe we defined earlier (using the internal LUSID Fx Forward
 discounting model). Also be aware as mentioned at the start of this section that we're no longer running an "inline" aggregation but actually
 aggregating the portfolio.


In [28]:
def run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, recipe, effective_at):
    aggregation_request = models.AggregationRequest(
        effective_at=effective_at.isoformat(),
        inline_recipe=recipe,
        metrics=[
            models.AggregateSpec(key='Holding/default/PV',
                                 op='Value')
        ]
    )

    return api_factory.build(lusid.api.AggregationApi).get_aggregation(scope=scope, code=portfolio,
                                                             aggregation_request=aggregation_request)


result_t = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, trade_date)


aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
])


,Name,Effective At,Value
0,Portfolio PV,01 Jun 2020,-0.799475


#### Our Portfolio Value as Markets Fluctuate

As time progresses during the life of our FX forward positions the markets will inevitably fluctuate. As they do we can update
the market data our pricing models consume to allow us to see the impact of those fluctuations on our portfolio value.

Let's simulate some movements over the coming days of our trade. We begin with the following data and an increase in the
GBP/USD spot rate:

In [32]:
t_p_1 = datetime(2020, 6, 2, tzinfo=pytz.utc)
upsert_gbp_usd_fx_rate(1.2550, t_p_1)

# run a portfolio valuation on day t+1 and compare with t
result_t_p_1 = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, t_p_1)

aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV', t_p_1, result_t_p_1.data[0]['Holding/default/PV']],
])

'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.255 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.7968127490039841 for 2020-06-02 00:00:00+00:00 uploaded to quote store.'

,Name,Effective At,Value
0,Portfolio PV,01 Jun 2020,-0.799475
1,Portfolio PV,02 Jun 2020,32.832566


Over the next couple of days US interest rates take a sudden shift and increase to 0.2% across the yield curve with a correponding
drop in the GBP/USD spot rate:

In [38]:
t_p_3 = datetime(2020, 6, 4, tzinfo=pytz.utc)

# increase in US interest rates
ois_curve_usd_json = load_ois_curve_json('USD', "shift interest rate higher")
upsert_interest_rate_curve(ois_curve_usd_json, market_data_scope, t_p_3, "USD/USDOIS")

# corresponding drop in fx rates
upsert_gbp_usd_fx_rate(1.2500, t_p_3)

# run valuation for day t+3 and compare with t+1 and t
result_t_p_3 = run_fx_forward_pricing_aggregation_on_portfolio(scope, portfolio, fx_forward_pricing_recipe, t_p_3)

aggregation_result_to_dataframe([
    ['Portfolio PV', trade_date, result_t.data[0]['Holding/default/PV']],
    ['Portfolio PV', t_p_1, result_t_p_1.data[0]['Holding/default/PV']],
    ['Portfolio PV', t_p_3, result_t_p_3.data[0]['Holding/default/PV']],
])


USD/USDOIS interest rate curve uploaded into scope=fx-forward-pricing-nb-market-data


'GBP/USD @ 1.2508 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.799488327470419 for 2020-06-01 00:00:00+00:00 uploaded to quote store.'

'GBP/USD @ 1.25 for 2020-06-04 00:00:00+00:00 uploaded to quote store.'

'USD/GBP @ 0.8 for 2020-06-04 00:00:00+00:00 uploaded to quote store.'

### Closing our FX Forward Position On Maturity

On maturity we now need to a transaction to close out our position which means we need to cover closing our long "stock"
holding in the forward instrument and then exchanging the contractual cash flows with an outflow in USD and and inflow in GBP

Ideally we'd like to wrap all those movements into one transaction type which would do the following :
* Apply direction=-1 Side1 movement removes our "stock" holding in the Forward instrument
* Apply direction=-1 Side2 cash movement to reflect our selling USD
* Apply direction=1 "?" cash movement to reflect our buying of GBP.


#### Defining a Custom Side for an Fx Forward Leg
As our Side2 movement deals in the outflow of USD we need a new custom side that handles the inflow of GBP. To see how we'll
create this custom side let's firstly take a look at how the Side2 movement works (Note : an understanding of Sides is required so please review [What is a Side in LUSID](https://support.finbourne.com/what-is-a-side-in-lusid) if necessary).

"Side 2" maps the settlement parameters of a transaction to a cash movement. That basically means that when we upsert a transaction and
generate a holding report the amount from the consideration will be added or subtracted from the settlement currency account. Based on that
our custom side needs to have the opposing cash movement impact. So it will take the transaction amount to be added or subtracted from the
transaction currency account when generating a holdings report.

In [43]:
def side_exists(side):
    response = system_configuration_api.list_configuration_transaction_types();
    return any(s.side==side for s in response.side_definitions)

def create_transaction_movement_side(side):
    if side_exists(side):
        display(f"{side} already exists.")
        return

    txn_cash_side_cfg = models.SideConfigurationDataRequest(
        side=side,
        # want to map the transaction amount to our tranaction currency account (in our exampl CCY_GBP)
        security="Txn:TradeCurrency",
        # the transaction currency
        currency="Txn:TradeCurrency",
        # the exchange rate used to map the transaction price to portfolio currency
        rate="Txn:TradeToPortfolioRate",
        # in our FX forward example the units match our transaction notional
        units="Txn:TradeAmount",
        # the transaction amount (i.e the BUY 10,000 GBP)
        amount="Txn:TradeAmount"
    )

    response = system_configuration_api.create_side_definition(side_configuration_data_request=txn_cash_side_cfg)

    if not response.side_definitions :
        raise StopExecution(f"Failed to create side. Response:{response}")

    display(f"Created {side} side.")


create_transaction_movement_side("FxFwd_TransactionCcy_Side")

'FxFwd_TransactionCcy_Side already exists.'

##### Creating a Transaction Type (Closing an FX Forward Position)

CLosing transaction type. depend how we want to model . For this example we'll close out forward position and transer cashflows
apply tansaction date immediately

Until now not actually defined side of the contract : were we long or shot dom vs fgn

two ways to cloese as diff movements on each. but in each case we close out the future


### Close if we've been long


In [44]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Long",
        description="Close a long position in an Fx Forward (Buy Dom, Sell Fgn)",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=-1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="Side2",
            direction=-1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="FxFwd_TransactionCcy_Side",
            direction=1
        )
    ],
)

if not create_txn_type_response.transaction_configs :
        raise StopExecution(f"Failed to create FxFwdClose_Long transaction type. Response:{create_txn_type_response}")

display("Created FxFwdClose_Long transaction type.")

'Created FxFwdClose_Long transaction type.'

### Close if we've been short

for display purposes only we would have flipped the signs

In [45]:
create_txn_type_response = create_transaction_type_configuration(
    api_factory,
    alias=models.TransactionConfigurationTypeAlias(
        type="FxFwdClose_Short",
        description="Close a short position in an Fx Forward (Sell Dom, Buy Fgn)",
        transaction_class="Basic",
        transaction_group="default",
        transaction_roles="Longer",
    ),
    movements=[
        models.TransactionConfigurationMovementDataRequest(
            movement_types="StockMovement",
            side="Side1",
            direction=-1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="Side2",
            direction=1
        ),
        models.TransactionConfigurationMovementDataRequest(
            movement_types="CashSettlement",
            side="FxFwd_TransactionCcy_Side",
            direction=-1
        )
    ],
)

if not create_txn_type_response.transaction_configs :
        raise StopExecution(f"Failed to create FxFwdClose_Short transaction type. Response:{create_txn_type_response}")

display("Created FxFwdClose_Short transaction type.")

'Created FxFwdClose_Short transaction type.'

In [46]:
fwd_units = 10000
contract_exc_rate = 1.2509
consideration = 10000
fgn_consideration = 10000 * 1.2509

close_fwd_request = models.TransactionRequest(
        transaction_id="fx_fwd_1w_close_001",
        type="FxFwdClose_Long",
        #type="FxFwdClose_FgnCashSettlementOnly",
        instrument_identifiers={"Instrument/default/ClientInternal": gbpusd_fx_fwd_1w_instr_client_id},
        transaction_date=maturity_dates['1W'].isoformat(),
        settlement_date=maturity_dates['1W'].isoformat(),
        units=units,
        transaction_price=models.TransactionPrice(price=contract_exc_rate, type="Price"),
        total_consideration=models.CurrencyAndAmount(amount=fgn_consideration, currency="USD"),
        exchange_rate=contract_exc_rate,
        transaction_currency="GBP",
    )

api_factory.build(lusid.api.TransactionPortfoliosApi).upsert_transactions(scope=scope,
                                                                                         code=portfolio,
                                                                                         transaction_request=[close_fwd_request])


{'href': 'https://khalid-local-dev.lusid.com/api/api/transactionportfolios/fx-forward-pricing-nb/fxfwd-portfolio-01/transactions?asAt=2020-07-10T08%3A05%3A19.4577830%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/portfolios/fx-forward-pricing-nb/fxfwd-portfolio-01?effectiveAt=2020-06-08T00%3A00%3A00.0000000%2B00%3A00&asAt=2020-07-10T08%3A05%3A19.4577830%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://khalid-local-dev.lusid.com/api/api/schemas/entities/UpsertPortfolioTransactionsResponse',
            'method': 'GET',
            'relation': 'EntitySchema'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'http://khalid-local-dev.lusid.com/app/insights/logs/0HM14BCUCIVBV:00000001',
            'method': 'GET',
            'relation

#### Evolution of Holdings During Life Cycle Of Maturity

View just bash balance

In [48]:
holdings_response_trade_date = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio, property_keys=["Instrument/default/Name"], effective_at=trade_date
)

display_holdings_summary(holdings_response_trade_date, trade_date)

,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-01 00:00:00+00:00,10000.000000,0.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"


In [50]:
holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio, property_keys=["Instrument/default/Name"], effective_at=t_p_3
)

display_holdings_summary(holdings_response_expiry, t_p_3)

,names,effectiveAt,units,settled_units,holding types,cost
0,GBP/USD 1W @ 1.2509,2020-06-04 00:00:00+00:00,10000.000000,0.000000,P,"{'amount': 10000.0, 'currency': 'GBP'}"


In [51]:
holdings_response_expiry = transaction_portfolios_api.get_holdings(
    scope=scope, code=portfolio, property_keys=["Instrument/default/Name"], effective_at=maturity_dates['1W']
)

display_holdings_summary(holdings_response_expiry, maturity_dates['1W'])



,names,effectiveAt,units,settled_units,holding types,cost
0,CCY_USD,2020-06-08 00:00:00+00:00,-12509.000000,-12509.000000,B,"{'amount': -12509.0, 'currency': 'USD'}"
1,CCY_GBP,2020-06-08 00:00:00+00:00,10000.000000,10000.000000,B,"{'amount': 10000.0, 'currency': 'GBP'}"
